In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *

from auctions.webutils import *

from scipy.stats import loguniform
import copy

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import analyze_checkpoint

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy


Loading BokehJS ...

In [3]:
# load experiment
experiments = ['jun2', 'jun2outcome']
runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()

# See also: jun2outcome for outcome sampling

In [36]:
# For each run, I want to know: ApproxNashConv, Auction Length, Revenue, Potential Function, Game Name, Game Mod

In [59]:
# #### Quick q: Are these games small enough to get NashConv? Yes: roughly 60 seconds. 10 seconds if the cache is primed!!!
# from open_spiel.python.algorithms import exploitability
# import time
# start = time.time()
# nc = exploitability.nash_conv(game, final_checkpoint.get_model())
# print(time.time() - start, nc)

In [5]:
game = runs[0].game.load_as_spiel()
policy = runs[0].equilibriumsolverruncheckpoint_set.last().get_model()


In [12]:
# Want to call this guy on each player
policy.action_probabilities?

Signature: policy.action_probabilities(state, player_id=None)
Docstring:
Returns the MCCFR average policy for a player in a state.

If the policy is not defined for the provided state, a uniform
random policy is returned.

Args:
  state: A `pyspiel.State` object.
  player_id: Optional, the player id for which we want an action. Optional
    unless this is a simultaneous state at which multiple players can act.

Returns:
  A `dict` of `{action: probability}` for the specified player in the
  supplied state. If the policy is defined for the state, this
  will contain the average MCCFR strategy defined for that state.
  Otherwise, it will contain all legal actions, each with the same
  probability, equal to 1 / num_legal_actions.
File:      /apps/open_spiel/open_spiel/python/algorithms/mccfr.py
Type:      method


In [60]:
game_cache = dict()

In [80]:
records = []
for run in tqdm(runs):
    record = dict(game_name=run.game.name, potential=run.config.get('potential_function', 'None'), seed=run.config.get('seed'), run_name=run.name)

    game = game_cache.get(run.game.name, run.game.load_as_spiel())
    game_cache[run.game.name] = game
    
    record['information_policy'] = InformationPolicy(game.auction_params.information_policy).name
    record['clock_speed'] = game.auction_params.increment
    record['base_game_name'] = '_'.join(run.game.name.split('/')[1].split('_')[:2]) # Stupid naming convention that will surely bite us later
    
    # Get the last eval
    final_checkpoint = run.equilibriumsolverruncheckpoint_set.last()
    
    # Get algorithm
    alg = run.config.get('solver_type', 'PPO')
    if alg == 'cfr':
        alg += '_' + run.config.get('sampling_method', '')
    record['alg'] = alg
    
    # Slowwwwww you may want to comment this out when not using it
    # start = time.time()
    # record['nash_conv'] = exploitability.nash_conv(game, final_checkpoint.get_model())
    # record['nash_conv_time'] = time.time() - start
    
    record['walltime'] = run.walltime()
    
    record = {**record, **analyze_checkpoint(final_checkpoint)}
    records.append(record)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:04<00:00, 18.82it/s]


In [81]:
df = pd.DataFrame.from_records(records)

In [85]:
df.groupby('alg')['walltime'].describe() / 3600

,count,mean,std,min,25%,50%,75%,max
alg,,,,,,,,
cfr_external,0.0125,1.977340,0.519881,1.128304,1.457960,2.115878,2.444383,2.810325
cfr_outcome,0.0125,1.057444,0.113848,0.859575,0.987953,1.063941,1.137879,1.246310


In [74]:
groupers = ['game_name', 'clock_speed']
df.sort_values(groupers).set_index([df.index]+groupers, drop=True)

,,,potential,seed,run_name,information_policy,base_game_name,alg,nash_conv,nash_conv_time,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,auction_lengths,common_allocations
,game_name,clock_speed,,,,,,,,,,,,,,,,
0,jun2/jun2_0_base.json,0.3,None,102,jun2_jun2_0_base-cfr_externalmccfr_external-102,SHOW_DEMAND,jun2_0,cfr_external,0.000105,61.309917,26.443607,55.562395,13.012805,32.000000,127.018808,87.562395,1.800420,"[((1, 1), 9996)]"
1,jun2/jun2_0_base.json,0.3,None,101,jun2_jun2_0_base-cfr_externalmccfr_external-101,SHOW_DEMAND,jun2_0,cfr_external,0.000156,9.523690,25.729028,49.370212,14.812425,38.251601,128.163265,87.621813,1.738095,"[((1, 1), 7466), ((1, 0), 2530)]"
2,jun2/jun2_0_base.json,0.3,None,100,jun2_jun2_0_base-cfr_externalmccfr_external-100,SHOW_DEMAND,jun2_0,cfr_external,0.000113,9.537872,26.800650,55.187146,12.958488,32.001901,126.948184,87.189047,1.734920,"[((1, 1), 9996), ((1, 0), 1)]"
3,jun2/jun2_0_base.json,0.3,None,101,jun2_jun2_0_base-cfr_outcomemccfr_outcome-101,SHOW_DEMAND,jun2_0,cfr_outcome,0.012956,9.214106,25.411659,49.724687,14.757392,38.219418,128.113155,87.944104,1.770592,"[((1, 1), 7491), ((1, 0), 2513)]"
4,jun2/jun2_0_base.json,0.3,None,102,jun2_jun2_0_base-cfr_outcomemccfr_outcome-102,SHOW_DEMAND,jun2_0,cfr_outcome,0.004786,9.556986,25.684760,49.328934,14.825567,38.332770,128.172031,87.661704,1.745302,"[((1, 1), 7440), ((1, 0), 2564)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,jun2/jun2_4_high_speed.json,0.6,None,102,jun2_jun2_4_high_speed-cfr_externalmccfr_exter...,SHOW_DEMAND,jun2_4,cfr_external,0.000128,8.796891,13.475280,51.000000,2.498499,51.000000,117.973779,102.000000,1.000000,"[((1, 1), 9992)]"
86,jun2/jun2_4_high_speed.json,0.6,None,101,jun2_jun2_4_high_speed-cfr_externalmccfr_exter...,SHOW_DEMAND,jun2_4,cfr_external,0.000169,8.947706,13.474580,50.994896,2.498499,51.000000,117.967974,101.994896,1.000000,"[((1, 1), 9991), ((0, 0), 1)]"
87,jun2/jun2_4_high_speed.json,0.6,None,101,jun2_jun2_4_high_speed-cfr_outcomemccfr_outcom...,SHOW_DEMAND,jun2_4,cfr_outcome,0.000932,8.553172,13.475280,51.000000,2.498199,50.994896,117.968375,101.994896,1.000000,"[((1, 1), 9992)]"


In [16]:
# TODO: Inner groupby on revenue, auction_length
df.groupby(groupers)['auction_lengths'].describe()[['min', 'max']]

,,min,max
game_name,clock_speed,,
jun2/jun2_0_base.json,0.3,1.734920,1.800420
jun2/jun2_0_hide_demand.json,0.3,1.734920,1.800420
jun2/jun2_0_high_speed.json,0.6,1.497349,1.738095
jun2/jun2_1_base.json,0.3,2.155862,2.324862
jun2/jun2_1_hide_demand.json,0.3,2.155862,2.324862
jun2/jun2_1_high_speed.json,0.6,1.375100,2.000000
jun2/jun2_2_base.json,0.3,1.739896,2.116635
jun2/jun2_2_hide_demand.json,0.3,1.739896,2.116635
jun2/jun2_2_high_speed.json,0.6,1.740348,1.750100


In [ ]:
Game.objects.get(name='regional_vs_national.json').

In [8]:
def delta(grp):
    lengths = grp.sort_values('potential')['auction_lengths'].values
    return lengths[1] - lengths[0]

valid_potentials = ['None', 'auction_length']
df.query('potential in @valid_potentials and run_name.str.contains("ppo_3")').groupby(['game_name', 'seed']).apply(delta)

,,game_name,potential,seed,run_name,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,auction_lengths,common_allocations
game_name,seed,,,,,,,,,,,,
